In [3]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr

In [4]:
symbol=pd.read_csv('EQUITY_L.csv')

In [5]:
symbol.head()

,SYMBOL,NAME OF COMPANY,Unnamed: 2,Unnamed: 3
0,20MICRONS,20 Microns Limited,NaN,NaN
1,21STCENMGM,21st Century Management Services Limited,NaN,NaN
2,3IINFOLTD,3i Infotech Limited,NaN,NaN
3,3MINDIA,3M India Limited,NaN,NaN
4,3PLAND,3P Land Holdings Limited,NaN,NaN


In [6]:
symbol=np.array(symbol)

In [7]:
type(symbol[0][0])

str

In [8]:
string=symbol[0][0]

In [9]:
string

'20MICRONS'

In [10]:
import datetime as dt
x=dt.date.today()

y=x-dt.timedelta(120)

In [11]:
x

datetime.date(2022, 10, 13)

In [12]:
(symbol.shape)[0]

1814

In [13]:
def reverse_hammer(data,i):
    
    open_close=(data[i][3]-data[i][4])/data[i][3]*100
    open_close=abs(open_close)
    
    low=(data[i][2]-data[i][3])/data[i][2]*100
    low=abs(low)
    high=(data[i][1]-data[i][4])/data[i][1]*100
    high=abs(high)
    # print(open_close," ",high," ",low)
    if(open_close*3<high and low*3<high):
        #print(data[i][0])
        return(1)
    return(-1)
    

In [14]:
def hammer(data,i):
    
    open_close=(data[i][3]-data[i][4])/data[i][3]*100
    open_close=abs(open_close)
    
    low=(data[i][2]-data[i][3])/data[i][2]*100
    low=abs(low)
    high=(data[i][1]-data[i][4])/data[i][1]*100
    high=abs(high)
    # print(open_close," ",high," ",low)
    if(open_close*3<low and high*3<low):
        #print(data[i][0])
        return(1)
    return(-1)

In [15]:
def harebozo(data,i):
    
    open_close=(data[i][3]-data[i][4])/data[i][3]*100
    open_close=abs(open_close)
    
    low=(data[i][2]-data[i][3])/data[i][2]*100
    low=abs(low)
    high=(data[i][1]-data[i][4])/data[i][1]*100
    high=abs(high)
    # print(open_close," ",high," ",low)
    high_low=(data[i][1]-data[i][2])/data[i][1]*100
    high_low=abs(high_low)
    if((open_close>high*4 and open_close>low*4) or (open_close>high*5 and data[i][3]>data[i][1]) or(open_close>low*5 and data[i][4]>data[i][2]) ):
        
        #print(data[i][0])
        return(1)
    return(-1)


In [16]:
def spin_top(data,i):
    
    open_close=(data[i][3]-data[i][4])/data[i][3]*100
    open_close=abs(open_close)
    
    low=(data[i][2]-data[i][3])/data[i][2]*100
    low=abs(low)
    high=(data[i][1]-data[i][4])/data[i][1]*100
    high=abs(high)
    # print(open_close," ",high," ",low)
    
    if(low>5*open_close and high>5*open_close and abs(high-low)<0.5):    
        #print(data[i][0])
        return(1)
    return(-1)
    

In [17]:
def func(data,curr):
    temp=curr
    dec=0
    inc=0
    while(curr>temp-5):
        if(data[curr][4]<data[curr-1][4]):
            dec=dec+1
        else:
            inc=inc+1
        curr=curr-1
    return(dec-inc)
        

In [18]:
def check_pattern(data,i):
     
    if(hammer(data,i)==1 or reverse_hammer(data,i)==1 or spin_top(data,i)==1 or harebozo(data,i)==1):
         if(func(data,i)>0):
             return(1)
         else:
             return(-1)
    return(0)        
    

In [19]:
def get_symbol(string,x,y):
    try:
        df=pdr.get_data_yahoo(string,start=x,end=y)
        return df
    except:
        try:
            df=pdr.get_data_yahoo(string+'.BO',start=x,end=y)
            return df
        except:
            try:
                df=pdr.get_data_yahoo(string+'.NS',start=x,end=y)
                return df
            except:
                df=pdr.get_data_yahoo('RELIANCE.NS',start=x,end=y)


In [20]:
def piv_high(data,temp):
    count=0
    sum=0
    for i in range(temp-1,temp):
        sum=sum+data[i][1]
        count=count+1
    avg=sum/count
    return avg
    

In [21]:
def piv_close(data,temp):
    count=0
    sum=0
    for i in range(temp-1,temp):
        sum=sum+data[i][4]
        count=count+1
    avg=sum/count
    return avg
    

In [22]:
def piv_low(data,temp):
    count=0
    sum=0
    for i in range(temp-1,temp):
        sum=sum+data[i][2]
        count=count+1
    avg=sum/count
    return avg
    

In [23]:
file=open('suggest.txt','w')
file.close()

In [24]:
len(symbol)

1814

In [25]:

file=open('suggest.txt','w')
i=0
result=[]
while(i<len(symbol)-1000):
    try:
        print("Index=",i)
        string=symbol[i][0]
        df=get_symbol(string,y,x)
        df=df.reset_index()
        
        print("\n")
        data=np.array(df)
        temp=(df.shape)[0]-1
        if(temp<4):
            i=i+1
            continue
        check=check_pattern(data,temp-1)
        print("check=",check)
        print("i=",i)
        print("check=",check)
        if(check>0):
            file.write("Company name=")
            file.write(symbol[i][1])
            file.write("\n")
            file.write("Suggestion=Buy stock\n")
            file.write("Entry price=")
            avg=data[temp-1][1]
            avg=avg+(avg*0.005)
            file.write(str(round(avg,2)))
            high=piv_high(data,temp-1)
            low=piv_low(data,temp-1)
            close=piv_close(data,temp-1)

            pp=(high+low+close)/3
            rr1=(2*pp)-low
            target1=rr1
            rr2=pp+(high-low)
            target2=rr2
            file.write("\nTarget price 1=")
            file.write(str(round(target1,2)))
            file.write("\nTarget price 1=")
            file.write(str(round(target2,2)))
          
            file.write("\nStop loss=")
            stop_loss=data[temp-1][2]
            stop_loss=stop_loss-(stop_loss*0.002)
            file.write(str(round(stop_loss,2)))
            file.write("\n\n")
           
            print("Company name=",symbol[i][1])
            print("Suggestion=Buy stock")
            print("Entry price=",avg)
            print("Target price1=",target1)
            print("Target price2=",target2)
            print("Stop loss",stop_loss)
            
            if(data[temp-1][4]<data[temp][4]):
                result.append([symbol[i][1],data[temp-1][0],'bullish','bullish'])
            else:
                result.append([symbol[i][1],data[temp-1][0],'bullish','bearish'])
            i=i+1
        elif check<0:
            file.write("Company name=")
            file.write(symbol[i][1])
            file.write("\n")
            file.write("Suggestion=Sell stock\n")
            file.write("Entry price=")
            avg=data[temp-1][1]
            avg=avg+(avg*0.005)
            file.write(str(round(avg,2)))
            high=piv_high(data,temp-1)
            low=piv_low(data,temp-1)
            close=piv_close(data,temp-1)

            pp=(high+low+close)/3
            rr1=(2*pp)-low
            target1=rr1
            rr2=pp+(high-low)
            target2=rr2
            file.write("\nTarget price 1=")
            file.write(str(round(target1,2)))
            file.write("\nTarget price 1=")
            file.write(str(round(target2,2)))
          
            file.write("\nStop loss=")
            stop_loss=data[temp-1][2]
            stop_loss=stop_loss-(stop_loss*0.002)
            file.write(str(round(stop_loss,2)))
            file.write("\n\n")
           
            print("Company name=",symbol[i][1])
            print("Suggestion=Sell stock")
            print("Target price1=",target1)
            print("Target price2=",target2)
            print("Stop loss",stop_loss)
            
            if(data[temp-1][4]<data[temp][4]):
                result.append([symbol[i][1],data[temp-1][0],'bearish','bullish'])
            else:
                result.append([symbol[i][1],data[temp-1][0],'bearish','bearish'])
            i=i+1
        else:
            i=i+1
    except:    
            
        
        
        i=i+1
file.close()
    

Index= 0


check= -1
i= 0
check= -1
Company name= 20 Microns Limited
Suggestion=Sell stock
Target price1= 100.26667022705078
Target price2= 101.48333740234375
Stop loss 96.65629847717285
Index= 1


check= 0
i= 1
check= 0
Index= 2


check= 0
i= 2
check= 0
Index= 3


check= 0
i= 3
check= 0
Index= 4


Index= 5


check= 0
i= 5
check= 0
Index= 6


check= 0
i= 6
check= 0
Index= 7


check= 0
i= 7
check= 0
Index= 8


check= -1
i= 8
check= -1
Company name= Aakash Exploration Services Limited
Suggestion=Sell stock
Target price1= 15.21666653951009
Target price2= 15.983333269755045
Stop loss 14.121699619293214
Index= 9


check= 0
i= 9
check= 0
Index= 10


check= -1
i= 10
check= -1
Company name= Aaron Industries Limited
Suggestion=Sell stock
Target price1= 167.43332926432294
Target price2= 171.66666158040366
Stop loss 159.68
Index= 11


check= 0
i= 11
check= 0
Index= 12


check= 0
i= 12
check= 0
Index= 13


Index= 14


check= 0
i= 14
check= 0
Index= 15


check= 0
i= 15
check= 0
Index= 16


Index= 1

In [26]:
res_df=pd.DataFrame(result,columns=['Date','Company Name','Prediction','Actual'])

In [27]:
res_df.to_csv('prediction_res.csv')

In [47]:
count=0
b=0
for i in range(len(result)):
    if(result[i][2]==result[i][3] and result[i][2]=='bullish'):
        count=count+1
    elif(result[i][2]==result[i][3] and result[i][2]=='bearish'):
        b=b+1

In [50]:
acq=(count+b)/len(result)

In [54]:
print("The accuraccy is=",round(acq,2))

The accuraccy is= 0.58
